In [1]:
exec(open("init_notebook.py").read())

current working dirc:\muktadir\junction-art


In [2]:
%load_ext autoreload
%autoreload 2

In [13]:
import dill
import os
import numpy as np
from junctionart.library.Configuration import Configuration
from junctionart.junctions.LaneConfigurationStrategies import LaneConfigurationStrategies
from junctionart.junctions.IntersectionValidator import IntersectionValidator
from junctionart.junctions.SequentialJunctionBuilder import SequentialJunctionBuilder
import junctionart.extensions as extensions
import pyodrx as pyodrx
import logging
from tqdm import tqdm
logging.basicConfig(level=logging.INFO)

In [21]:
class SequentialIntersectionExporter():

    def __init__(self):
        
        self.configuration = Configuration()
        self.outputDir= os.path.join(os.getcwd(), 'output')
        lastId = 0
        self.seed = 2
        self.builder = SequentialJunctionBuilder(
                                                    minAngle=np.pi/10, 
                                                    maxAngle=np.pi * .75,
                                                    straightRoadLen=5, 
                                                    probLongConnection=0.5,
                                                    probMinAngle=0.5,
                                                    probRestrictedLane=0.2,
                                                    maxConnectionLength=50,
                                                    minConnectionLength=20,
                                                    random_seed=self.seed)
        
        self.randomState =self.configuration.get("random_state")
        self.validator = IntersectionValidator()

        pass
    
    def gen(self,
        maxNumberOfRoadsPerJunction = 3,
        minLanePerSide = 1,
        maxLanePerSide = 2,
        numberOfIntersections = 10):

        created = 0
        intersections = []
        # create 
        with tqdm(total=numberOfIntersections) as pbar:
            while created < numberOfIntersections:
                path = self.configuration.get("harvested_straight_roads")
                intersection = self.builder.createWithRandomLaneConfigurations(path, 
                                    0, 
                                    maxNumberOfRoadsPerJunction=maxNumberOfRoadsPerJunction, 
                                    maxLanePerSide=maxLanePerSide, 
                                    minLanePerSide=minLanePerSide, 
                                    internalConnections=True, 
                                    cp1=pyodrx.ContactPoint.end,
                                    internalLinkStrategy = LaneConfigurationStrategies.SPLIT_ANY,
                                    getAsOdr=False)


#                 odr = intersection.odr
                # xmlPath = f"output/test_createWithRandomLaneConfigurations-split-any-{maxNumberOfRoadsPerJunction}-{sl}.xodr"
#                 xmlPath = f"output/seed-{self.seed}-{maxNumberOfRoadsPerJunction}-way-{sl}.xodr"
#                 odr.write_xml(xmlPath)
                isValid = self.validator.validateIncidentPoints(intersection, self.builder.minConnectionLength)
                if isValid:
                    intersections.append(intersection)
                    created += 1
                    pbar.update(1)

                if created % 1000 == 0:
                    print(f"generated {created}")
        
        
        return intersections
    
    def save(self, outputPath, intersections):
        with open(outputPath, 'wb') as handler:
            print(f"writing out {len(intersections)}")
            dill.dump(intersections, handler)
    
    def genNSave(self, 
        maxNumberOfRoadsPerJunction = 3,
        minLanePerSide = 1,
        maxLanePerSide = 2,
        numberOfIntersections = 10):
        
        intersections = self.gen(
            maxNumberOfRoadsPerJunction = maxNumberOfRoadsPerJunction,
            minLanePerSide = minLanePerSide,
            maxLanePerSide = maxLanePerSide,
            numberOfIntersections = numberOfIntersections
        )
        
        laneStr = ""
        if minLanePerSide == maxLanePerSide and minLanePerSide == 1:
            laneStr = "-2lane"
        
        outputPath = os.path.join(self.outputDir, f"Sequential-intersections{laneStr}-{numberOfIntersections}.dill")

        self.save(outputPath, intersections)
        

                    


In [19]:
exporter = SequentialIntersectionExporter()

In [20]:
exporter.genNSave()

100%|██████████████████████████████████████████████████████████████████████████████████| 10/10 [00:00<00:00, 27.78it/s]


writing out 10
None
